# Deploy Infrastructure

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os, sys, json
sys.path.insert(1, '../shared')  # add the shared directory to the Python path
import utils


deployment_name = 'ai-foundry'
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
resource_group_location = "eastus" # all the resources will be deployed in this location

build = 0

# https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/concepts/models
models_config = [{"name": "DeepSeek-R1", "publisher": "DeepSeek", "version": "1", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "Phi-4", "publisher": "Microsoft", "version": "3", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "gpt-4o", "publisher": "OpenAI", "version": "2024-05-13", "sku": "GlobalStandard", "capacity": 1},
                 {"name": "text-embedding-3-large", "publisher": "OpenAI", "version": "1", "sku": "Standard", "capacity": 1}
                 ]

searchservice_sku = "standard"
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "./src/weather/mcp-server"

principalId = ""
if not principalId:
    principalId = !az ad signed-in-user show --query id -o tsv
    principalId = principalId[0]
    
utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 20:36:16.037705 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 20:36:18.351818 :2s]
👉🏽 Current user: pablocastao@microsoft.com
👉🏽 Tenant ID: 16b3c013-d300-468d-ac64-7eda0820b6d3
👉🏽 Subscription ID: 06d043e2-5a2e-46bf-bf48-fffee525f377


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "modelsConfig": { "value": models_config },
        "principalId": { "value": principalId }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-ai-foundry-bancolombia 
👉🏽 Resource group lab-ai-foundry-bancolombia does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-ai-foundry-bancolombia --location eastus --tags source=ai-gateway 
✅ Resource group 'lab-ai-foundry-bancolombia' created ⌚ 20:36:26.074235 :3s]
⚙️ Running: az deployment group create --name ai-foundry-bancolombia --resource-group lab-ai-foundry-bancolombia --template-file main.bicep --parameters params.json 
⛔ Deployment 'ai-foundry-bancolombia' failed ⌚ 20:47:52.407033 :26s] WARNING: A new Bicep release is available: v0.34.44. Upgrade now by running "az bicep upgrade".
c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\1-infra\modules\add-capability-host.bicep(24,20) : Warning BCP081: Resource type "Microsoft.MachineLearningServices/workspaces@2024-10-01-preview" does not have types available. Bicep is unable to validate resource properties pr

<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [4]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    project = utils.get_deployment_output(output, 'projectConnectionString', 'Project Connection String')    
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    weather_containerapp_url = utils.get_deployment_output(output, 'weatherMCPServerContainerAppFQDN', 'Weather Container App URL')
    bingConnectionName = utils.get_deployment_output(output, 'bingConnectionName', 'Bing Connection Name')
    model_deployment_name = utils.get_deployment_output(output, 'model_deployment_name', 'Model Deployment Name')

    # Write the project and outputs to an .env file
    with open('../.env', 'w') as env_file:
        env_file.write(f"PROJECT_CONNECTION_STRING='{project}'\n")
        env_file.write(f"APPLICATION_INSIGHTS_NAME='{app_insights_name}'\n")
        env_file.write(f"CONTAINER_REGISTRY_NAME='{container_registry_name}'\n")
        env_file.write(f"WEATHER_CONTAINERAPP_RESOURCE_NAME='{weather_containerapp_resource_name}'\n")
        env_file.write(f"WEATHER_CONTAINERAPP_URL='{weather_containerapp_url}'\n")
        env_file.write(f"BING_CONNECTION_NAME='{bingConnectionName}'\n")
        env_file.write(f"MODEL_DEPLOYMENT_NAME='{model_deployment_name}'\n")
        env_file.write(f"AOAI_API_VERSION='2024-12-01-preview'\n")


⚙️ Running: az deployment group show --name ai-foundry-bancolombia -g lab-ai-foundry-bancolombia 
✅ Retrieved deployment: ai-foundry-bancolombia ⌚ 20:49:22.219590 :5s]
⛔ Failed to retrieve output property: 'projectConnectionString'
Error: 'NoneType' object is not subscriptable ⌚ 20:49:22.221593 


Exception: Failed to retrieve output property: 'projectConnectionString'
Error: 'NoneType' object is not subscriptable

### 4️⃣ Build and deploy the MCP Servers

In [ ]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

⚙️ Running: az acr build --image weather-mcp-server:v0.1 --resource-group lab-ai-foundry --registry acr5yrvw4ytnjmmm --file ./src/weather/mcp-server/Dockerfile ./src/weather/mcp-server/. --no-logs 
✅ Weather MCP Server image was successfully built ⌚ 16:29:00.406055 :8s]
⚙️ Running: az containerapp update -n aca-weather-5yrvw4ytnjmmm -g lab-ai-foundry --image "acr5yrvw4ytnjmmm.azurecr.io/weather-mcp-server:v0.1" 
✅ Weather MCP Server deployment succeeded ⌚ 16:29:20.737501 :20s]
